<a href="https://colab.research.google.com/github/newh08/AI_Voyage99/blob/main/question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge nltk evaluate rouge_score bert_score sacrebleu peft bitsandbytes accelerate dataset

import nltk
nltk.download('punkt')

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from google.colab import files

uploaded = files.upload()


Saving ItemData.json to ItemData.json


In [3]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import random
from typing import List, Tuple
import torch
from datasets import Dataset
from transformers import AutoTokenizer

model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_path)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# JSON 파일 로드
with open("ItemData.json", "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

# DataFrame 변환
df = pd.DataFrame(data)
print(df.info())

# 데이터 분할 (90% train, 10% validation)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

schema = '''Table : "customers" Column: "customer_id", "first_name", "last_name", "email", "phone_number", "address_id",
    foreign_key: "address_id" from "addresses", "address_id",
    foreign_key: "customer_id" from "orders", "customer_id" [SEP]
Table : "addresses" Column: "address_id", "street", "city", "state", "zip_code", "country_id",
    foreign_key: "country_id" from "countries", "country_id" [SEP]
Table : "orders" Column: "order_id", "customer_id", "order_date", "total_amount", "status",
    foreign_key: "customer_id" from "customers", "customer_id",
    foreign_key: "order_id" from "order_items", "order_id" [SEP]
Table : "order_items" Column: "order_item_id", "order_id", "product_id", "quantity", "price",
    foreign_key: "order_id" from "orders", "order_id",
    foreign_key: "product_id" from "products", "product_id" [SEP]
'''

def create_prompt(question: str) -> str:
    return f"""Generate an SQL query using the schema below.
    Schema: {schema}
    Question: {question}
    SQL Qurey:
    """

def create_vals_and_labels(df: pd.DataFrame):
    list_prompts = df["question"].apply(create_prompt).tolist()
    list_labels = df["label"].tolist()
    return list_prompts, list_labels

# 학습 데이터셋
prompts_train, labels_train = create_vals_and_labels(train_df)

# 검증 데이터셋
prompts_val, labels_val = create_vals_and_labels(val_df)

print(f"📌 Train 데이터 개수: {len(prompts_train)}")
print(f"📌 Validation 데이터 개수: {len(prompts_val)}")

# ✅ Hugging Face Dataset 변환
train_dataset = Dataset.from_pandas(pd.DataFrame({"text": prompts_train, "labels": labels_train}))
val_dataset = Dataset.from_pandas(pd.DataFrame({"text": prompts_val, "labels": labels_val}))

# 데이터셋 토크나이징
def preprocess_function(examples):
    # 입력 데이터 토크나이징
    model_inputs = tokenizer(
        examples["text"],
        padding="max_length",  # ✅ `max_length`로 길이 통일
        truncation=True,
        max_length=512,  # ✅ 최대 길이 설정 (필요에 따라 조정)
        return_tensors="pt"
    )

    # 정답 데이터 토크나이징
    labels = tokenizer(
        examples["labels"],
        padding="max_length",  # ✅ `max_length`로 입력과 동일하게 맞춤
        truncation=True,
        max_length=512,  # ✅ 입력과 같은 max_length 유지
        return_tensors="pt"
    )

    # `labels["input_ids"]`에서 패딩 토큰을 -100으로 변경하여 Trainer가 무시하도록 설정
    labels["input_ids"][labels["input_ids"] == tokenizer.pad_token_id] = -100

    # PyTorch 텐서로 변환 후 `.to(device)` 적용
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# 데이터셋 토크나이징 및 MPS 이동
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

tokenized_train.set_format("torch")
tokenized_val.set_format("torch")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440 entries, 0 to 439
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  440 non-null    object
 1   label     440 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB
None
📌 Train 데이터 개수: 396
📌 Validation 데이터 개수: 44


Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

In [4]:
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback
import evaluate
import numpy as np
from datasets import Dataset
import pandas as pd
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM
import wandb

wandb.init(project="sql_test", name="DeepSeek_1.5B")

model_o = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # 메모리 절약을 위해 float16 적용
    low_cpu_mem_usage=True  # ✅ CPU에서 메모리 최적화
).to("cpu")


# LoRA 설정 (r=8, lora_alpha=32, lora_dropout=0.1)
lora_config = LoraConfig(
    r=4,  # LoRA 차원 (작을수록 메모리 절약)
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.1,  # 과적합 방지 드롭아웃
    target_modules=["q_proj", "v_proj"],  # 학습할 레이어 선택
    bias="none"
)

model = get_peft_model(model_o, lora_config).to(device)

del model_o
torch.cuda.empty_cache()

# ✅ Hugging Face의 accelerate 라이브러리를 사용해 원본 모델을 CPU에 유지
from accelerate import dispatch_model, infer_auto_device_map


# ✅ 자동으로 device_map 생성
device_map = infer_auto_device_map(model, max_memory={0: "30GB", "cpu": "64GB"})

model = dispatch_model(model, device_map=device_map)  # ✅ LoRA Adapter만 GPU로 올림

# BLEU 및 ROUGE 메트릭 로드
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

def compute_metrics(pred):
    predictions, labels = pred

    # 확률값(logits)이라면 argmax 적용
    if isinstance(predictions, tuple):
        predictions = predictions[0]  # logits 선택
    predictions = np.argmax(predictions, axis=-1)  # 가장 높은 확률의 토큰 선택

    # NumPy 배열 또는 Tensor를 리스트로 변환
    predictions = predictions.tolist() if isinstance(predictions, np.ndarray) else predictions
    labels = labels.tolist() if isinstance(labels, np.ndarray) else labels

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 계산
    bleu_score = bleu_metric.compute(predictions=decoded_preds, references=[[l] for l in decoded_labels])
    # ROUGE 계산
    rouge_score = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "eval_bleu": bleu_score["bleu"],
        "eval_rouge1": rouge_score["rouge1"],
        "eval_rouge2": rouge_score["rouge2"],
        "eval_rougeL": rouge_score["rougeL"],
    }



training_args = TrainingArguments(
    output_dir='hf_transformer',  # 모델, log 등을 저장할 directory
    num_train_epochs=2,  # epoch 수
    per_device_train_batch_size=1,  # training data의 batch size
    per_device_eval_batch_size=1,  # validation data의 batch size
    logging_strategy="steps",  # steps 끝날 때마다 training loss 등을 log하라는 의미
    logging_steps=10,  # 매 10 스텝마다 로깅
    do_train=True,  # 학습을 진행하겠다는 의미
    do_eval=True,  # 학습 중간에 validation data에 대한 평가를 수행하겠다는 의미
    evaluation_strategy="epoch",  # 매 epoch가 끝날 때마다 validation data에 대한 평가를 수행한다는 의미
    save_strategy="epoch",  # 매 epoch가 끝날 때마다 모델을 저장하겠다는 의미
    learning_rate=1e-5,  # optimizer에 사용할 learning rate
    load_best_model_at_end=True,  # 학습이 끝난 후, validation data에 대한 성능이 가장 좋은 모델을 채택하겠다는 의미
    metric_for_best_model="bleu",      # BLEU 점수를 기준으로 best model 선정
    greater_is_better=True
)

accuracy = evaluate.load("accuracy")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: newh08 (newh008) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-4-c5439a31c37a>:95: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,12.054700,No log


early stopping required metric_for_best_model, but did not find eval_bleu so early stopping is disabled


KeyError: "The `metric_for_best_model` training argument is set to 'eval_bleu', which is not found in the evaluation metrics. The available evaluation metrics are: []. Consider changing the `metric_for_best_model` via the TrainingArguments."

In [10]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.2
    Uninstalling transformers-4.48.2:
      Successfully uninstalled transformers-4.48.2


In [8]:
import gc
import torch

import torch

torch.cuda.empty_cache()  # 캐시 정리
torch.cuda.reset_max_memory_allocated()  # 최대 메모리 할당 초기화


# ✅ 모든 불필요한 텐서 삭제
gc.collect()  # Python의 Garbage Collector 실행
torch.cuda.empty_cache()  # CUDA 캐시 메모리 정리
torch.cuda.ipc_collect()  # 공유 메모리 정리

import os

# PyTorch 메모리 최적화 설정
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 설정 확인
print(os.environ["PYTORCH_CUDA_ALLOC_CONF"])



expandable_segments:True


In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM
import torch
import wandb



# 모델 로드
model_path = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 모델 및 토크나이저 다운로드
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

model.to(device)

# 🔹 모델을 사용하여 SQL 예측
def generate_sql(model, tokenizer, questions):
    sql_outputs = []
    for question in questions:
        inputs = tokenizer(question, return_tensors="pt").to(device)  # 입력 토큰화
        outputs = model.generate(**inputs, max_length=512)  # SQL 생성
        result = tokenizer.decode(outputs[0], skip_special_tokens=True)  # 디코딩
        sql = result.split("SQL Qurey:", 1)[1].strip()
        sql_outputs.append(sql)
    return sql_outputs

# 검증 데이터 SQL 생성
predicted_sqls = generate_sql(model, tokenizer, prompts_val)

# 결과 샘플 확인
for i in range(3):
    print(f"🔹 질문: {prompts_val[i]}")
    print(f"🔹 실제 SQL: {labels_val[i]}")
    print(f"🔹 모델 예측 SQL: {predicted_sqls[i]}\n")

from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(predicted_sqls, labels_val, avg=True)

print(f"✅ 학습 전 ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"✅ 학습 전 ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"✅ 학습 전 ROUGE-L: {scores['rouge-l']['f']:.4f}")

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# BLEU 점수 계산
smoothie = SmoothingFunction().method1
bleu_score = corpus_bleu([[label.split()] for label in labels_val],
                         [pred.split() for pred in predicted_sqls],
                         smoothing_function=smoothie)

print(f"✅ 학습 전 BLEU Score: {bleu_score:.4f}")



Using device: cuda


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


KeyboardInterrupt: 

In [ ]:
!ps aux | grep python

!kill -9 1951


root          85  0.9  0.0      0     0 ?        Z    06:07   0:13 [python3] <defunct>
root          86  0.0  0.0  63632 50964 ?        S    06:07   0:00 python3 /usr/local/bin/colab-file
root         135  0.4  0.1 866264 124220 ?       Sl   06:07   0:05 /usr/bin/python3 /usr/local/bin/j
root        1951 10.3 13.5 65019760 11888528 ?   Ssl  06:15   1:34 /usr/bin/python3 -m colab_kernel_
root        1995  0.2  0.0 545760 21368 ?        Sl   06:15   0:02 /usr/bin/python3 /usr/local/lib/p
root        3638  0.1  0.0 1274540 44684 ?       Ssl  06:20   0:00 /usr/local/lib/python3.11/dist-pa
root        3656  0.0  0.0 977292 21984 ?        Sl   06:20   0:00 /usr/local/lib/python3.11/dist-pa
root        6172  0.0  0.0   7376  3412 ?        S    06:30   0:00 /bin/bash -c ps aux | grep python
root        6174  0.0  0.0   6484  2264 ?        S    06:30   0:00 grep python


In [ ]:
# 학습 후 재평가
predicted_sqls_after_train = generate_sql(model, tokenizer, prompts_val)

scores = rouge.get_scores(predicted_sqls_after_train, labels_val, avg=True)

print(f"✅ 학습 후 ROUGE-1: {scores['rouge-1']['f']:.4f}")
print(f"✅ 학습 후 ROUGE-2: {scores['rouge-2']['f']:.4f}")
print(f"✅ 학습 후 ROUGE-L: {scores['rouge-l']['f']:.4f}")


bleu_score = corpus_bleu([[label.split()] for label in labels_val],
                         [pred.split() for pred in predicted_sqls_after_train],
                         smoothing_function=smoothie)

print(f"✅ 학습 후 BLEU Score: {bleu_score:.4f}")


✅ 학습 후 ROUGE-1: 0.7020
✅ 학습 후 ROUGE-2: 0.5192
✅ 학습 후 ROUGE-L: 0.6976
✅ 학습 후 BLEU Score: 0.3974
